#  Image2Map

Illustrative examples with a Kohonen's Self-Organizing Map.

In [ ]:
import os, sys
PATH = os.path.dirname(os.path.abspath("."))
sys.path.append(os.path.join(PATH, "src"))
%load_ext autoreload
%autoreload 2

___

## Initialize model

Initializes an instance of the `SOM` class.

The following hyperparameters have been used for the examples in this notebook:

* MNIST: `epochs=10, k_units=100, k_shape=(10,10), topology='GRID'`
* Lines: `epochs=100, k_units=8, k_shape=8, topology='LINE'`

In [ ]:
import image2map as im

som = im.SOM(
    k_units=8,               # LINES
    k_shape=8,               # LINES
    topology="line",         # LINES
    # k_units=100,             # MNIST
    # k_shape=(10, 10),        # MNIST
    # k_units=k*k,             # Number of units (neurons) in the SOM.
    # k_shape=(k, k),          # Shape of the output map.
    # k_dist="l2",             # Distance among units ('l1', 'l2', 'chebyshev').
    # n_inputs=None,           # Number of input features.
    # n_shape=None,            # Shape of the input data.
    # topology="grid",         # Topology of the SOM ('GRID', 'MESH', 'LINE', 'RING').
    # unit_topology="square",  # Type of tiling for the SOM ('SQUARE', 'HEX').
    # radius_max=None,         # Maximum radius for neighborhood function.
    # radius_min=0,            # Minimum radius for neighborhood function.
    # radius_rate=None,        # Decay function type ('exp', 'lin').
    # radius_decay="exp",      # Decay rate (constant).
    # alpha_max=0.1,           # Initial learning rate.
    # alpha_min=0.01,          # Final learning rate.
    # alpha_rate=None,         # Decay function type ('exp', 'lin').
    # alpha_decay="exp",       # Decay rate (constant).
    # phi="lap",               # Neighborhood function type ('lap', 'exp', 'sqd', 'lin').
    # k=1.0,                   # Scaling factor.
    # sigma=1.0,               # Standard deviation.
    # seed=42                    # Random seed.
)
som

___

## Load dataset

Load input data and split into training and test sets.

In [ ]:
# X = ...
# X_train, X_test = im.utils.split_ts(X, size=0.8)

##### Example: [MNIST](https://www.tensorflow.org/datasets/catalog/mnist) dataset

In [ ]:
X_train, y_train, X_test, y_test = im.utils.load_mnist(root="../data/input/mnist")
fig, ax = im.subplots(X_train[:10])  # Visualize the weights for the first 10 training samples.
fig

##### Example: Lines in a 2-d space

In [ ]:
X = im.utils.load_images("../data/input/images_100_16x16_png")
X_train, X_test = im.utils.split_ts(X, size=0.8)
fig, ax = im.subplots(X)
fig

##### Example: Points in a 2-d space

In [ ]:
# import numpy as np
# from itertools import product
# X, d = [], 10
# for i, j in product(range(d), range(d)):
#     x = np.ones((d, d))
#     x[i, j] = -1
#     X.append(x)
# X = np.array(X).reshape(-1, d*d)
# X_train, X_test = im.utils.split_ts(X, size=0.8)
# fig, ax = im.subplots(X)
# fig

### Normalize features

Scale input features in a specified range, by default set to $[-1, 1]$.

In [ ]:
wmin, wmax = (-1, 1)
X_train = im.utils.minmax_scale(X_train, (wmin, wmax))
X_test = im.utils.minmax_scale(X_test, (wmin, wmax))
# X = im.utils.minmax_scale(X, scale)

___

## Fit model to data

With the data loaded, either `fit` the model to match input `X` feature dimensionality or manually define its attributes.

In [ ]:
som.fit(X_train)
# som.n_inputs = n*n    # Number of input features.
# som.n_shape = (n, n)  # Input data dimensions.
som

#### Initialize neurons and weights

Randomly initialize weights in the interval $(w_{min}, w_{max})$ and position neurons in the output map depending on the selected topological properties.

> Neuron positions and weight vectors are automatically initialized on `train` if undefined; this cell is only included to allow plotting initial weights next.

In [ ]:
som.init_neurons()
som.plot_neurons(j0=0, r=0)
# som.Y  # Neuron positions..

In [ ]:
som.init_weights(wmin, wmax)
som.plot_weights(j0=0, r=0)
# som.W  # Neuron weights.

___
## Train model

#### Set path to save weights

In [ ]:
weights_path = "weights.npy"  # Path to save weights (final only)

Saving the weights at every epoch can be achieved by setting the path to a folder instead:

In [ ]:
weights_path = "weights/"     # Path to save weights (every epoch)
# !mkdir -p {weights_path}      # Create directory for weights

Alternatively, to load the weights from previously trained model:

In [ ]:
# som.load_weights("/path/to/weights.npy")

#### Begin model training

In [ ]:
epochs = 10
som.train(X_train, epochs=epochs, save_weights=weights_path)

In [ ]:
# som.reset_epoch()   # Resets current epoch.
# som.reset_alpha()   # Resets current learning rate self.alpha.
# som.reset_radius()  # Resets current iteration radius self.radius.
# som.init_weights()  # Re-initializes weights.

#### Plot final weights

Renders map of neuron positions and their weights.

In [ ]:
som.plot_weights(output="mnist.eps")

#### Plot learning animation

Load weights saved to disk into an array and animate learning process over time.

In [ ]:
!pip install --quiet ipympl
%matplotlib ipympl

Z = []
for epoch in range(epochs):
    som.load_weights("{}weights_epoch_{}.npy".format(weights_path, epoch))
    Z.append(som.W)

nrows, ncols = som.k_shape if type(som.k_shape) == tuple else (1, som.k_shape)
im.animate(Z, nrows=nrows, ncols=ncols, output="animation.gif", interval=30)

## Predict new data

Find the best matching unit to an unseen test sample.

In [ ]:
i = 0
fig, ax = im.plot_image(X_test[i], figsize=(2,2))
fig

In [ ]:
j0 = som.predict(X_test[i:i+1], bmu=True)[0]
fig = som.plot_weights(j0=j0, r=None)
fig

___

## Unit tests

In [ ]:
%load_ext autoreload
%autoreload 2
from image2map import tests

In [ ]:
tests.plot_grid_mesh_fig(output="fig-grid-mesh.eps")

In [ ]:
tests.plot_decay_fig(xmin=0, xmax=1, output="fig-decay.eps")

In [ ]:
tests.plot_phi_fig(r=10, output="fig-phi.eps")

In [ ]:
tests.plot_dist_fig(j0=45, r=2.5, output="fig-dist.eps")